Import packages and processed run metrics

In [2]:
# packages
import os
import re
import numpy as np
import pandas as pd

# source
helper = __import__('00_helper')

# control
project_names = [
    'femnist--s02', 'sst2', 'pubmed',
    #'cifar--alpha5.0', 'cifar--alpha0.5', 'cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

df = helper.load_data(project_names)


input datasets:
	 (413, 35)
	 (1200, 44)
	 (780, 37)
all runs: (2393, 44)


Filter data to runs of interest
Apply summary function to filtered dataset

In [3]:
subset_df = df
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches

subset_df = subset_df.loc[(subset_df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', subset_df.shape)

subset_df = subset_df.loc[(subset_df.K < 6) | (subset_df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', subset_df.shape)

subset_df = subset_df.loc[((10 * subset_df.beta).astype('Int64') % 2 == 0) | (subset_df.method != 'exact')]
print('\t reduce beta grid for decay:', subset_df.shape)

## column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if (
        not re.search('test', name)
        and not re.search('f1', name)
        and not re.search('loss', name)
    )
]]

def convert_SI(x):
    if pd.isna(x):
        return x

    SI = {
        'K': 10e3,
        'M': 10e6,
        'G': 10e9,
        'T': 10e12
    }

    units = re.sub('[0-9.]', '', x)
    x = float(re.sub('[^0-9.]', '', x))

    if units == '':
        return x
    return x * SI[units]

#convert_SI('19.0M')

input dataset: (2393, 44)
	 reduce to 3 or fewer local update steps: (1787, 44)
	 reduce to 3 or fewer local meta-learning steps for pfedme: (1707, 44)
	 reduce beta grid for decay: (1610, 44)


In [4]:
subset_df['sys_avg/total_model_size'].unique()

array(['12.53M', '6.27M', '18.8M', nan, '4.1M', '12.3M', '8.2M', '33.45K',
       '66.9K', '100.35K'], dtype=object)

In [5]:
## get best runs for each group
# regardless of finetuning
id_columns = ['dataset', 'method']
metrics = [name for name in subset_df.columns if re.search('^sys', name)]
print(metrics)
object_metrics= [
    name for name, types in zip(subset_df.columns, subset_df.dtypes)
    if re.search('^sys', name)
    and types == 'object'
]

float_df = subset_df.loc[subset_df.finetune == 0]
float_df = float_df[id_columns + metrics]
float_df[object_metrics] = float_df[object_metrics].applymap(convert_SI).apply(pd.to_numeric)
float_df['sys_avg/total_bytes'] = float_df['sys_avg/total_download_bytes'] \
                          + float_df['sys_avg/total_upload_bytes']


['sys_avg/global_convergence_time_minutes', 'sys_avg/total_model_size', 'sys_avg/fl_end_time_minutes', 'sys_avg/total_flops', 'sys_avg/local_convergence_time_minutes', 'sys_avg/local_convergence_round', 'sys_avg/global_convergence_round', 'sys_avg/total_upload_bytes', 'sys_avg/total_download_bytes']


In [6]:
sub_metrics = [
    'sys_avg/global_convergence_round',
    'sys_avg/global_convergence_time_minutes',
    'sys_avg/total_bytes',
    'sys_avg/total_flops'
]
avg_runs = float_df.groupby(id_columns)[sub_metrics].mean()
avg_runs

sys_avg/global_convergence_round  \
dataset      method                                     
femnist--s02 ditto                         195.161290   
             exact                         142.000000   
             fedavg                        101.000000   
             fedbn                         450.000000   
             fedem                         213.333333   

                     sys_avg/global_convergence_time_minutes  \
dataset      method                                            
femnist--s02 ditto                                 44.421018   
             exact                                 22.467354   
             fedavg                                13.829542   
             fedbn                                 23.969965   
             fedem                                117.686957   

                     sys_avg/total_bytes  sys_avg/total_flops  
dataset      method                                            
femnist--s02 ditto          1.385956e+07         3.349287e+12  
             exact          8.720425e+06         2.506085e+12  
             fedavg         7.401090e+06         1.965300e+12  
             fedbn          1.782730e+07         2.701800e+12  
             fedem          3.126813e+07         5.058616e+13